#Install Dependencies

In [1]:
!gdown https://drive.google.com/uc?id=13q11ZnhYehDtibrHCvA7kfSkCKcxMpcv

import zipfile
with zipfile.ZipFile("fr-domain-classification.zip","r") as zip_ref:
    zip_ref.extractall("")
    

Downloading...
From: https://drive.google.com/uc?id=13q11ZnhYehDtibrHCvA7kfSkCKcxMpcv
To: /content/fr-domain-classification.zip
259MB [00:02, 125MB/s]


In [2]:
!pip install langdetect

     |████████████████████████████████| 983kB 7.3MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993193 sha256=22a1fe9baaeede462950aa268e087be82331f3a53e40609e5c1e7a4fd87017f2
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d40a60ea06e6d669e8e5012e6c8b947a57
Successfully built langdetect


In [3]:
!pip install translate

     |████████████████████████████████| 174kB 11.0MB/s 
     |████████████████████████████████| 81kB 9.6MB/s 
     |████████████████████████████████| 4.6MB 40.6MB/s 
     |████████████████████████████████| 276kB 52.8MB/s 
     |████████████████████████████████| 102kB 11.7MB/s 
     |████████████████████████████████| 573kB 45.7MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.3-cp36-cp36m-linux_x86_64.whl size=44229 sha256=4c69ddead5566831284c24ee84c5f2346d67a3c155ae1cf17e154f1f46f17abb
  Stored in directory: /root/.cache/pip/wheels/e4/76/4d/a95b8dd7b452b69e8ed4f68b69e1b55e12c9c9624dd962b191
  Created wheel for distlib: filename=distlib-0.3.0-cp36-none-any.whl size=340429 sha256=6d08b084377642e3b9a7cabdbc625704011da21d9f7384e1e974ba4aff0ac436
  Stored in directory: /root/.cache/pip/wheels/6e/e8/db/c73dae4867666e89ba3cfbc4b5c092446f0e584eda6f409cbb
Successfully built pyyaml distlib
ERROR: pytest 3.6.4 has requirement pluggy<0.8,>=0.5, but you'll have pluggy 0.13.1 which is incompatible

#Load dependencies

In [4]:
import os
import numpy as np
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
import codecs
from os import path
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings("ignore")
import gensim 
from gensim.models import Word2Vec
import numpy as np
from langdetect import detect
from translate import Translator
from nltk.stem.snowball import FrenchStemmer
import re
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.stem.snowball import SnowballStemmer
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
#List Of English And French Stopwords
stopwords =stopwords.words('french') + stopwords.words('english')
#English And French Stemmer
stemmer_french = FrenchStemmer()
stemmer_english = SnowballStemmer("english")

#A RegexpTokenizer Splits A String Into Substrings Using A Regular Expression.
tokenizer = RegexpTokenizer(r'\w+')
# Translator To Translate The Text Into French
translator = Translator(to_lang="fr")


#Load The Data

In [0]:
# Read training data
with open("train.csv", 'r') as f:
    train_data = f.read().splitlines()

train_hosts = list()
y_train = list()
for row in train_data:
    host, label = row.split(",")
    train_hosts.append(host)
    y_train.append(label.lower())

# Read test data
with open("test.csv", 'r') as f:
    test_hosts = f.read().splitlines()

# Load the textual content of a set of webpages for each host. 
# The encoding parameter is required since the majority of our text is french.
train_data = list()
for host in train_hosts:
    try :
        with open('text/text/' + host,encoding="utf-8") as f: 
                text = f.read().replace("\n", "").lower()
    except :
        with open('text/text/' + host,encoding="latin-1") as f: 
                text = f.read().replace("\n", "").lower()
    train_data.append(text)

# Get textual content of web hosts of the test set
test_data = list()
for host in test_hosts:
    try :
        with open('text/text/' + host,encoding="utf-8") as f: 
                text = f.read().replace("\n", "").lower()
    except :
        with open('text/text/' + host,encoding="latin-1") as f: 
                text = f.read().replace("\n", "").lower()
    test_data.append(text)   

#Text Preprocessing

In [0]:
def text_preprocessing (string) : 

    output = string.replace("_"," ")
    #Remove non-English characters, by selecting characters not in a given ASCII range 
    output=re.sub(r'[\u4e00-\u9fff]+'," ", output)

    output = ''.join(c for c in output if not c.isdigit())

    #A tokenizer that splits a string using a regular expression, which matches either the tokens or the separators between tokens.

    word_tokens = tokenizer.tokenize(output)
    text = []
    for w in word_tokens:
        if w not in stopwords and len(w)>2:
            text.append(w)
            
    #Stemmers remove morphological affixes from words, leaving only the word stem.
    text = list(map(lambda x :stemmer_french.stem(x),text))
    text = list(map(lambda x :stemmer_english.stem(x),text))

    #The Treebank tokenizer uses regular expressions to tokenize text as in Penn Treebank. 
    return TreebankWordDetokenizer().detokenize(text) 

In [0]:
#Pytranslate is a state-of-art, well-equipped center, providing total solutions to customers in all translation, data conversion and pre-press operations like translation, multilingual typesetting and content management.
#http://www.pytranslate.com/
def translate(tokens) :
    
    string = TreebankWordDetokenizer().detokenize(tokens)
    
    if len(string) >2 :
        #Port of Nakatani Shuyo's language-detection  to detect the language of the text.
        if not detect(string) == 'fr' :
            text = tokenizer.tokenize(translator.translate(string))
        else :
            text = tokenizer.tokenize(string)
    else :
        text = tokenizer.tokenize(string)
        
    return text

In [0]:
#Translate And Tokenize Training Data
train_data = list(map(text_preprocessing,train_data))

In [0]:
#Translate And Tokenize Test Data
test_data = list(map(text_preprocessing,test_data))

# Term Frequency–Inverse Document Frequency,

In [0]:
#Convert a collection of raw documents to a matrix of TF-IDF features.
vec = TfidfVectorizer(decode_error='ignore', strip_accents='unicode', min_df=1, max_df=500,sublinear_tf=True,stop_words=stopwords)
X_train = vec.fit_transform(train_data)

# Create the test matrix following the same approach as in the case of the training matrix
X_test = vec.transform(test_data)

# Train The Model

In [12]:
#Linear classifiers (SVM, logistic regression, a.o.) with SGD training.
#https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html

clf = SGDClassifier(loss='log', penalty='l2', tol=None, learning_rate= 'optimal', eta0=0.15, class_weight= None, alpha= 0.00001)
#Fit linear model with Stochastic Gradient Descent.
clf.fit(X_train, y_train)

 	

#Predict class labels for samples in X_train.
print(clf.score(X_train, y_train))




0.9411764705882353


# Load Pretrained Node Embeddings Matrix 

In [13]:
!gdown https://drive.google.com/uc?id=1E6K2bW5I8X0zIrXmQRy-c02-lZVKMl6n

Downloading...
From: https://drive.google.com/uc?id=1E6K2bW5I8X0zIrXmQRy-c02-lZVKMl6n
To: /content/pretrained_embeddings.npy
57.4MB [00:00, 183MB/s]


#Categorical Encoding Using Label Encoding

In [0]:
#Encode target labels with value between 0 and n_classes-1.
#This transformer should be used to encode target values, i.e. y, and not the input X.
#https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y_train)
y_train_num = le.transform(y_train)

#Train Node   And Test Node

In [0]:
embeddings=np.load("pretrained_embeddings.npy")
idx=[int(u) for u in train_hosts]
idx_test=[int(u) for u in test_hosts]

# Train The Model

In [16]:
from sklearn.svm import SVC
#C-Support Vector Classification.
#https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

classifier = SVC(kernel='poly',degree=4,probability=True)
#Fit the SVM model according to the given training data.
classifier.fit(embeddings[idx], y_train_num)

#Return the mean accuracy on the given test data and labels.
print(classifier.score(embeddings[idx], y_train_num))



0.8649411764705882


# Weighted Average

In [0]:
#A hyperparameter for the Weighted Average
alpha=0.6
#The final Prediction
y_pred=((1-alpha)*classifier.predict_proba(embeddings[idx_test])+(alpha)*clf.predict_proba(X_test))

# Kaggle submission

In [0]:
# Write predictions to a file
with open('text_baseline_'+str(alpha)+'.csv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    lst = clf.classes_.tolist()
    lst.insert(0, "Host")
    writer.writerow(lst)
    for i,test_host in enumerate(test_hosts):
        lst = y_pred[i,:].tolist()
        lst.insert(0, test_host)
        writer.writerow(lst)